In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

In [29]:
data_train = pd.read_csv('data/salary-train.zip')
data_test = pd.read_csv('data/salary-test-mini.csv')
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [30]:
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True).str.lower()
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True).str.lower()

In [31]:
vectorizer = TfidfVectorizer(min_df=5)
X_train_description = vectorizer.fit_transform(data_train['FullDescription'])
X_test_description = vectorizer.transform(data_test['FullDescription'])

In [32]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

In [33]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [37]:
X_train = hstack([X_train_description, X_train_categ])
y_train = data_train['SalaryNormalized']
clf = Ridge(alpha=1, random_state=241)
clf.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

In [51]:
pred = clf.predict(hstack([X_test_description, X_test_categ]))
' '.join([f'{i:.2f}' for i in pred])

'56555.62 37188.32'